To create a program that allows the user to pick 4 edges and then automatically finds the paper edges and dewarps the page, you can use OpenCV for image processing and a simple GUI for user interaction. Here's a step-by-step implementation:

### Step 1: Set Up Environment

Ensure you have the following libraries installed:



In [ ]:
# pip install opencv-python-headless numpy



### Step 2: Implement the Dewarping Program

Create a Python script, e.g., `dewarp_page.py`:



In [ ]:
import cv2
import numpy as np
import pytesseract
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Global variables to store points
points = []

def click_event(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow("Image", image)
        if len(points) == 4:
            cv2.destroyAllWindows()

def get_paper_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            return approx
    return None

def apply_perspective_transform(image, points):
    pts = np.array(points, dtype="float32")
    rect = np.zeros((4, 2), dtype="float32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def main():
    global image
    image_path = 'path_to_your_image.jpg'
    image = cv2.imread(image_path)
    cv2.imshow("Image", image)
    cv2.setMouseCallback("Image", click_event)
    cv2.waitKey(0)

    if len(points) == 4:
        dewarped_image = apply_perspective_transform(image, points)
        cv2.imshow("Dewarped Image", dewarped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Please select exactly 4 points.")

if __name__ == "__main__":
    main()



### Step 3: Run the Program

Replace `'path_to_your_image.jpg'` with the path to your image and run the script:



In [ ]:
python dewarp_page.py



This script will load the image, allow the user to click on 4 points to define the corners of the page, and then apply a perspective transformation to dewarp the page. The dewarped image will be displayed in a new window.

Similar code found with 2 license types